In [ ]:
import torch
import torch.nn.functional as F

num_samples = 5  # 你想要进行的采样次数
all_logits = []  # 用于收集所有采样的logits
all_entropies = []  # 用于收集所有采样的信息熵

for _ in range(num_samples):
    standard_nor_distribution = torch.randn(scores_mean.shape).to(scores_mean.device)
    scores = scores_mean + torch.mul(standard_nor_distribution, torch.sqrt(scores_cov))
    bs, head, seqlen, _ = scores.size()
    scores.masked_fill_(mask == 0, -1e32)
    scores = F.softmax(scores, dim=-1)
    scores = self.attdrop(scores)
    v_ = torch.matmul(scores, v)
    concat = v_.transpose(1, 2).contiguous().view(bs, -1, self.d_model)
    output = self.out_proj(concat)
    logits = self.out(torch.cat([output], dim=-1)).squeeze(-1)
    all_logits.append(logits)
    probs = torch.sigmoid(logits[s >= 0])  # Convert logits to probabilities
    entropy = -probs * torch.log(probs) - (1 - probs) * torch.log(1 - probs)  # Binary entropy
    all_entropies.append(entropy)

# 计算信息熵的平均值
mean_entropy = torch.mean(torch.stack(all_entropies), dim=0)

# 计算多次预测平均值的信息熵
mean_logits = torch.mean(torch.stack(all_logits), dim=0)
mean_probs = torch.sigmoid(mean_logits[s >= 0])
mean_entropy_of_mean = -mean_probs * torch.log(mean_probs) - (1 - mean_probs) * torch.log(1 - mean_probs)

# 计算差值
entropy_diff = mean_entropy - mean_entropy_of_mean

# 计算加权损失
all_losses = []
masked_labels = s[s >= 0].float()
for logits in all_logits:
    masked_logits = logits[s >= 0]
    pred_loss = F.binary_cross_entropy_with_logits(masked_logits, masked_labels, reduction='none')
    weighted_loss = pred_loss * entropy_diff  # 加权损失
    all_losses.append(weighted_loss.mean())  # 对每个样本的加权损失取平均

# 累加损失
total_loss = sum(all_losses) / num_samples
